In [ ]:
def wordfreq(l_ticker, groupby_variable):
    '''
    inputs:
    df = df_cell files
    groupby_variable = variable to group words by 
    
    '''

    finalITEMS = pd.DataFrame()
    finalNONQUANT = pd.DataFrame()
    
    
    for ticker in l_ticker: 
        l_files = get_company_files(s3_client, ticker=ticker, filename_pattern='^df_cells')
        for file in l_files:
            obj = s3_client.get_object(Bucket='esgnie', Key=file)
            df = pd.read_csv(obj['Body'])
            header_rows, nonquantrows, items, df2 = quants_parser(df)
            
            
            finalITEMS = finalITEMS.append(items)
            finalNONQUANT = finalNONQUANT.append(nonquantrows)

    
    
    items = commonwords(finalITEMS)
    nonquant = commonwords(finalNONQUANT)
    overall = items.rename(columns={"freq": "freq_quant_row", "proportion": "proportion_quant_row"})
    
    
    freq_nonquant_row = []
    proportion_nonquant_row = []

    for word in items.word:
        if word in list(nonquant.word):
            freq_nonquant_row.append(int(nonquant[nonquant.word == word].freq))
            proportion_nonquant_row.append(float(nonquant[nonquant.word == word].proportion))
        else:
            freq_nonquant_row.append(0)
            proportion_nonquant_row.append(0)
        
    overall["freq_nonquant_row"] = freq_nonquant_row
    overall["proportion_nonquant_row"] = proportion_nonquant_row
    overall["freq_diff"] = np.array(overall.freq_quant_row) - np.array(freq_nonquant_row)
    overall["proportion_diff"] = np.array(overall.proportion_quant_row) - np.array(proportion_nonquant_row)
    
    overall.sort_values(by=['proportion_diff'], ascending = False, inplace = True)
    